# [네이버 영화 리뷰 워드클라우드](https://blog.naver.com/111ambition/222836972997)

[Python Pandas : dropna (NaN value가 있는 row/column 제거하기)](https://cosmosproject.tistory.com/308)

## 필요한 모듈 및 데이터 불러오기

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import urllib.request
# !pip install konlpy → 아래 코드 오류에 대한 선행 코드
from konlpy.tag import Okt
from tqdm import tqdm
from wordcloud import WordCloud, STOPWORDS

In [2]:
# 데이터 불러오기
# urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt", filename="pydata/ratings_train.txt")
# urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt", filename="pydata/ratings_test.txt")

In [2]:
train_data = pd.read_table('pydata/ratings_train.txt') # read_table(): csv, txt 파일을 불러오는 함수
test_data = pd.read_table('pydata/ratings_test.txt')
train_data[:5]

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [3]:
test_data[:5]

,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,GDNTOPCLASSINTHECLUB,0
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0


In [4]:
print("train 셋 행 개수: {}개 / test 셋 행 개수: {}개".format(len(train_data), len(test_data)))

train 셋 행 개수: 150000개 / test 셋 행 개수: 50000개


## Stopwords 및 특수문자 필터링을 위한 준비

In [5]:
comment_words = ""
stopwords = set(STOPWORDS)

tokenized_text = []

## 문장 데이터를 단어화하기

In [6]:
for val in train_data["document"]:
    
    val = str(val) # 문장을 string으로 만들기
    tokens = val.split() # 문장을 쪼개기
    tokenized_text.append(tokens)
    comment_words += " ".join(tokens) + " "

In [16]:
train_data["document"]

0                                       아 더빙.. 진짜 짜증나네요 목소리
1                         흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나
2                                         너무재밓었다그래서보는것을추천한다
3                             교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정
4         사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...
                                ...                        
149995                                  인간이 문제지.. 소는 뭔죄인가..
149996                                        평점이 너무 낮아서...
149997                      이게 뭐요? 한국인은 거들먹거리고 필리핀 혼혈은 착하다?
149998                          청춘 영화의 최고봉.방황과 우울했던 날들의 자화상
149999                             한국 영화 최초로 수간하는 내용이 담긴 영화
Name: document, Length: 150000, dtype: object

In [9]:
# document 열의 중복 제거
train_data.drop_duplicates(subset= ['document'], inplace=True)

In [14]:
train_data = train_data.dropna(how = 'any') # Null 값이 존재하는 행 제거(any: row 또는 column에 Null 값이 한 개만 있어서 drop)
print(train_data.isnull().values.any()) # Null 값이 존재하는지 확인(any: 일부, all: 전부)

False


In [28]:
# 한글과 공백을 제외하고 모두 제거
train_data['document'] = train_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣+]", "")
train_data[:1]

<ipython-input-28-a6abd7bf468b>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  train_data['document'] = train_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣+]", "")


,id,document,label
0,9976970,아더빙진짜짜증나네요목소리,0
